USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print(f"다음 기기로 학습 : {device}")

In [7]:
!pip install konlpy

In [8]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from collections import Counter
from sklearn.model_selection import train_test_split

In [9]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x1696b384550>)

In [10]:
train_data = pd.read_table("ratings_train.txt")
test_data = pd.read_table("ratings_test.txt")

In [11]:
train_data.drop_duplicates(subset=["document"], inplace=True)
train_data["document"] = train_data["document"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)
train_data["document"] = train_data["document"].str.replace("^ +", "", regex=True)
train_data["document"] = train_data["document"].replace("", np.nan)
train_data = train_data.dropna(how = "any")

In [12]:
test_data.drop_duplicates(subset = ["document"], inplace=True)
test_data["document"] = test_data["document"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)
test_data["document"] = test_data["document"].str.replace("^ +", "", regex=True)
test_data["document"] = test_data["document"].replace("", np.nan)
test_data = test_data.dropna(how="any")

In [13]:
print('전처리 샘플의 개수 :',len(train_data),len(test_data))

전처리 샘플의 개수 : 145393 48852


In [14]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
okt = Okt()
X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    X_train.append(stopwords_removed_sentence)

100%|██████████| 145393/145393 [03:49<00:00, 632.26it/s]


In [15]:
with open('X_train.pickle', 'wb') as f:
    pickle.dump(X_train, f, pickle.HIGHEST_PROTOCOL)

In [16]:
X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

100%|██████████| 48852/48852 [01:27<00:00, 555.61it/s]


In [17]:
with open('X_test.pickle', 'wb') as f:
    pickle.dump(X_test, f, pickle.HIGHEST_PROTOCOL)

In [18]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0, stratify=y_train)

In [19]:
word_list = []
for sent in X_train:
    for word in sent:
      word_list.append(word)
word_counts = Counter(word_list)

In [20]:
print('총 단어수 :', len(word_counts))
print('훈련 데이터에서의 단어 영화의 등장 횟수 :', word_counts['영화'])
print('훈련 데이터에서의 단어 공감의 등장 횟수 :', word_counts['공감'])

총 단어수 : 88276
훈련 데이터에서의 단어 영화의 등장 횟수 : 40264
훈련 데이터에서의 단어 공감의 등장 횟수 : 784


In [21]:
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
print('등장 빈도수 상위 10개 단어')
print(vocab[:10])

등장 빈도수 상위 10개 단어
['영화', '너무', '정말', '만', '적', '진짜', '으로', '로', '점', '에서']


In [22]:
threshold = 3
total_cnt = len(word_counts) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

for key, value in word_counts.items():
    total_freq = total_freq + value
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 88276
등장 빈도가 2번 이하인 희귀 단어의 수: 60078
단어 집합에서 희귀 단어의 비율: 68.05700303593277
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 5.644509532507002


In [23]:
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
vocab_size = total_cnt - rare_cnt
vocab = vocab[:vocab_size]
print('단어 집합의 크기 :', len(vocab))

단어 집합의 크기 : 28198


In [24]:
word_to_index = {}
word_to_index['<PAD>'] = 0
word_to_index['<UNK>'] = 1
for index, word in enumerate(vocab) :
  word_to_index[word] = index + 2
vocab_size = len(word_to_index)
print('패딩 토큰과 UNK 토큰을 고려한 단어 집합의 크기 :', vocab_size)
print('단어 <PAD>와 맵핑되는 정수 :', word_to_index['<PAD>'])
print('단어 <UNK>와 맵핑되는 정수 :', word_to_index['<UNK>'])
print('단어 영화와 맵핑되는 정수 :', word_to_index['영화'])

패딩 토큰과 UNK 토큰을 고려한 단어 집합의 크기 : 28200
단어 <PAD>와 맵핑되는 정수 : 0
단어 <UNK>와 맵핑되는 정수 : 1
단어 영화와 맵핑되는 정수 : 2


In [25]:
def texts_to_sequences(tokenized_X_data, word_to_index):
  encoded_X_data = []
  for sent in tokenized_X_data:
    index_sequences = []
    for word in sent:
      try:
          index_sequences.append(word_to_index[word])
      except KeyError:
          index_sequences.append(word_to_index['<UNK>'])
    encoded_X_data.append(index_sequences)
  return encoded_X_data

In [26]:
encoded_X_train = texts_to_sequences(X_train, word_to_index)
encoded_X_valid = texts_to_sequences(X_valid, word_to_index)
encoded_X_test = texts_to_sequences(X_test, word_to_index)

In [27]:
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value] = key
decoded_sample = [index_to_word[word] for word in encoded_X_train[0]]
print('기존의 첫번째 샘플 :', X_train[0])
print('복원된 첫번째 샘플 :', decoded_sample)

기존의 첫번째 샘플 : ['이야', '어쩜', '이렇게나', '지루할수가']
복원된 첫번째 샘플 : ['이야', '어쩜', '이렇게나', '지루할수가']


In [28]:
max_len = 30
def pad_sequences(sentences, max_len):
  features = np.zeros((len(sentences), max_len), dtype=int)
  for index, sentence in enumerate(sentences):
    if len(sentence) != 0:
      features[index, :len(sentence)] = np.array(sentence)[:max_len]
  return features
padded_X_train = pad_sequences(encoded_X_train, max_len=max_len)
padded_X_valid = pad_sequences(encoded_X_valid, max_len=max_len)
padded_X_test = pad_sequences(encoded_X_test, max_len=max_len)
print('훈련 데이터의 크기 :', padded_X_train.shape)
print('검증 데이터의 크기 :', padded_X_valid.shape)
print('테스트 데이터의 크기 :', padded_X_test.shape)

훈련 데이터의 크기 : (116314, 30)
검증 데이터의 크기 : (29079, 30)
테스트 데이터의 크기 : (48852, 30)


## LSTM을 사용한 감정 분석

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [30]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print(f"다음 기기로 학습 : {device}")

다음 기기로 학습 : cuda


In [31]:
train_label_tensor = torch.tensor(np.array(y_train))
valid_label_tensor = torch.tensor(np.array(y_valid))
test_label_tensor = torch.tensor(np.array(y_test))
print(train_label_tensor[:5])

tensor([0, 1, 1, 1, 1])


In [32]:

# - 문장 길이 = 500
# - 배치 크기 = 32
# - 데이터 개수 = 2만

# - LSTM 매개변수
  # - 단어 벡터의 차원 = 100
  # - LSTM의 은닉층의 크기 = 128
  # - 분류하고자 하는 카테고리 개수 = 2개

### 텍스트 분류를 위한 레이어

In [33]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
    super(TextClassifier, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    embedded = self.embedding(x)
    lstm_put, (hidden, cell) = self.lstm(embedded)
    last_hidden = hidden.squeeze(0)
    logits = self.fc(last_hidden)
    return logits

### 데이터를 텐서로 변경

In [34]:
encoded_train = torch.tensor(padded_X_train).to(torch.int64)
train_dataset = torch.utils.data.TensorDataset(encoded_train, train_label_tensor)  #문제와 정답지로 분류
train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=32)

encoded_valid = torch.tensor(padded_X_valid).to(torch.int64)
valid_dataset = torch.utils.data.TensorDataset(encoded_valid, valid_label_tensor)  #문제와 정답지로 분류
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, shuffle=True, batch_size=32)

encoded_test = torch.tensor(padded_X_test).to(torch.int64)
test_dataset = torch.utils.data.TensorDataset(encoded_test, test_label_tensor)  #문제와 정답지로 분류
test_dataloader = torch.utils.data.DataLoader(test_dataset, shuffle=True, batch_size=32)

### 배치데이터 구축

In [35]:
total_batch = len(train_dataloader)
print(total_batch)

3635


### 모델 생성

In [36]:
embedding_dim = 100
hidden_dim = 128
output_dim = 2
learning_rate = 0.01
num_epochs = 10

model = TextClassifier(vocab_size, embedding_dim, hidden_dim, output_dim)
model.to(device)

TextClassifier(
  (embedding): Embedding(28200, 100)
  (lstm): LSTM(100, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)

### optimizer 등 설정

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### 정확도 계산

In [38]:
def cacluate_accuracy(logits, labels):
  predicted = torch.argmax(logits, dim=1)
  correct = (predicted == labels).sum().item()
  total = labels.size(0)
  accuracy = correct / total
  return accuracy 

### 학습 및 계산

In [39]:
#학습할때 > 모델,valid_data, criterion(loss-function), device
def evlaute(model, valid_dataloader, criterion, device):
  val_loss = 0
  val_correct = 0
  val_total = 0
  model.eval()
  with torch.no_grad():
    for batch_X, batch_y in valid_dataloader:
      batch_X, batch_y = batch_X.to(device), batch_y.to(device)
      logits = model(batch_X)
      loss = criterion(logits, batch_y)
      val_loss += loss.item()
      val_correct += cacluate_accuracy(logits, batch_y) * batch_y.size(0)
      val_total += batch_y.size(0)
  val_accuracy = val_correct / val_total
  val_loss /= len(valid_dataset)
  return val_loss, val_accuracy

In [40]:
best_val_loss = float('inf')
#for epoch in range(num_epochs):
for epoch in range(5):
  train_loss = 0
  train_correct = 0
  train_total = 0
  model.train()
  for batch_X, batch_y in valid_dataloader:
    batch_X, batch_y = batch_X.to(device), batch_y.to(device)
    logits = model(batch_X)
    loss = criterion(logits, batch_y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
    train_correct += cacluate_accuracy(logits, batch_y) * batch_y.size(0)
    train_total += batch_y.size(0)
  train_accuracy = train_correct / train_total
  train_loss /= len(train_dataloader)
  val_loss, val_accuracy = evlaute(model, valid_dataloader, criterion, device)
  print(f"Epoch : {epoch + 1} / {num_epochs}")
  print(f"Train Loss : {train_loss} / Train Accuracy : {train_accuracy}")
  print(f"Valid Loss : {val_loss} / Valid Accuracy : {val_accuracy}")
  if val_loss < best_val_loss:
    best_val_loss = val_loss
    torch.save(model.state_dict(), "best_model_checkpoint.pt")


Epoch : 1 / 10
Train Loss : 0.17349517812755116 / Train Accuracy : 0.5032841569517521
Valid Loss : 0.021563869211933564 / Valid Accuracy : 0.5107465868840056
Epoch : 2 / 10
Train Loss : 0.14724115469432764 / Train Accuracy : 0.6751607689397847
Valid Loss : 0.013211023508672263 / Valid Accuracy : 0.816671825028371
Epoch : 3 / 10
Train Loss : 0.09963692655015681 / Train Accuracy : 0.8222772447470683
Valid Loss : 0.009337355079794033 / Valid Accuracy : 0.8821486295952405
Epoch : 4 / 10
Train Loss : 0.07436489161344802 / Train Accuracy : 0.8776092712954365
Valid Loss : 0.006837850340934669 / Valid Accuracy : 0.9205612297534304
Epoch : 5 / 10
Train Loss : 0.054911961547897147 / Train Accuracy : 0.9132707452113209
Valid Loss : 0.004727285914974985 / Valid Accuracy : 0.9480724921764847


### 모델 불러오기

### 측정